In [3]:
from skimage import transform, filters, exposure
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from sklearn.cross_validation import train_test_split
from keras.layers.convolutional import Conv2D

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
import os
import glob
import pandas as pd
import math
from PIL import Image
import numpy as np
from scipy.misc import imread, imsave, imresize
from natsort import natsorted
from scipy.misc import imread
import random
random.seed( 42 ) 

# Path of data files
path = "/home/ivan/python/julia/data"
 
# Input image dimensions
img_rows, img_cols = 32, 32
 
 

### Images preprocessing ###

for setType in ["test","train"]:
    # We have to make sure files are sorted according to labels, even if they don't have trailing zeros
    files = natsorted(glob.glob(path + "/"+setType+"/*"))
    """
    if setType == 'test':
        X_test = np.array([np.array(Image.open(fname)) for fname in files])
    else:
        X_train = np.array([np.array(Image.open(fname)) for fname in files])
    """
    if setType == 'test':
        X_test = np.array([np.array(imread(fname, flatten=True)) for fname in files])
    else:
        X_train = np.array([np.array(imread(fname, flatten=True)) for fname in files])
X_test=X_test[:,:,:,np.newaxis]
X_train=X_train[:,:,:,np.newaxis]
X_train /= 255
X_test /= 255
X_test.shape
    

(6220, 20, 20, 1)

In [6]:
def label2int(ch):
    asciiVal = ord(ch)
    if(asciiVal<=57): #0-9
        asciiVal-=48
    elif(asciiVal<=90): #A-Z
        asciiVal-=55
    else: #a-z
        asciiVal-=61
    return asciiVal
    
def int2label(i):
    if(i<=9): #0-9
        i+=48
    elif(i<=35): #A-Z
        i+=55
    else: #a-z
        i+=61
    return chr(i)
# Load labels
y_train = pd.read_csv("/home/ivan/python/julia/trainLabels.csv").values[:,1] #Keep only label
 
# Convert labels to one-hot vectors
Y_train = np.zeros((y_train.shape[0], len(np.unique(y_train))))
 
for i in range(y_train.shape[0]):
    Y_train[i][label2int(y_train[i])] = 1 # One-hot
Y_train_all = Y_train.copy()
X_train_all = X_train.copy()

In [7]:
X_train, X_val, Y_train, Y_val = \
        train_test_split(X_train_all, Y_train_all, test_size=0.2, stratify=np.argmax(Y_train_all, axis=1))

In [8]:
nb_classes = 62 # A-Z, a-z and 0-9
nb_epoch = 500
img_rows, img_cols = 20, 20
model = Sequential()

#model.add(Convolution2D(3, 3, 128,  border_mode='same', init='he_normal', activation = 'relu', input_shape=(1, img_rows, img_cols)))
model.add(Conv2D(3, (3, 32), kernel_initializer="he_normal", activation="relu", input_shape=(img_rows, img_cols, 1), padding="same"))
model.add(Conv2D(3, (3, 32), kernel_initializer="he_normal", activation="relu", padding="same"))

model.add(MaxPooling2D(data_format="channels_last", pool_size=(2, 2)))

model.add(Conv2D(3, (3, 64), kernel_initializer="he_normal", activation="relu", padding="same"))
#model.add(Conv2D(3, (3, 256), kernel_initializer="he_normal", activation="relu", padding="same"))

model.add(MaxPooling2D(data_format="channels_last", pool_size=(2, 2)))

model.add(Conv2D(3, (3, 128), kernel_initializer="he_normal", activation="relu", padding="same"))
model.add(Conv2D(3, (3, 256), kernel_initializer="he_normal", activation="relu", padding="same"))
#model.add(Conv2D(3, (3, 512), kernel_initializer="he_normal", activation="relu", padding="same"))

model.add(MaxPooling2D(data_format="channels_last", pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(2048, kernel_initializer="he_normal", activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(2048, kernel_initializer="he_normal", activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, kernel_initializer="he_normal", activation="softmax"))

In [58]:
from keras.models import Model
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout
inputs = Input((img_rows, img_cols, 1))
conv1 = Conv2D(3, (3, 32), activation="relu", padding="same", data_format='channels_first')(inputs)
conv1 = Conv2D(3, (3, 32), activation="relu", padding="same", data_format='channels_first')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)


conv2 = Conv2D(3, (3, 64), activation="relu", padding="same", data_format='channels_first')(pool1)
conv2 = Conv2D(3, (3, 64), activation="relu", padding="same", data_format='channels_first')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(3, (3, 128), activation="relu", padding="same", data_format='channels_first')(pool2)
conv3 = Conv2D(3, (3, 128), activation="relu", padding="same", data_format='channels_first')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

up4 = merge([UpSampling2D(size=(2, 2))(conv3), conv2], mode='concat', concat_axis=1)
conv4 = Conv2D(3, (3, 64), activation="relu", padding="same", data_format='channels_first')(up4)
conv4 = Conv2D(3, (3, 64), activation="relu", padding="same", data_format='channels_first')(conv4)

up5 = merge([UpSampling2D(size=(2, 2))(conv4), conv1], mode='concat', concat_axis=1)
conv5 = Conv2D(3, (3, 32), activation="relu", padding="same", data_format='channels_first')(up5)
conv5 = Conv2D(3, (3, 32), activation="relu", padding="same", data_format='channels_first')(conv5)


"""
conv4 = Conv2D(3, (3, 256), activation="relu", padding="same", data_format='channels_first')(pool3)
conv4 = Conv2D(3, (3, 256), activation="relu", padding="same", data_format='channels_first')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(3, (3, 512), activation="relu", padding="same", data_format='channels_first')(pool4)
conv5 = Conv2D(3, (3, 512), activation="relu", padding="same", data_format='channels_first')(conv5)

up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=1)
conv6 = Conv2D(3, (3, 256), activation="relu", padding="same", data_format='channels_first')(up6)
conv6 = Conv2D(3, (3, 256), activation="relu", padding="same", data_format='channels_first')(conv6)

up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=1)
conv7 = Conv2D(3, (3, 128), activation="relu", padding="same", data_format='channels_first')(up7)
conv7 = Conv2D(3, (3, 128), activation="relu", padding="same", data_format='channels_first')(conv7)

up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=1)
conv8 = Conv2D(3, (3, 64), activation="relu", padding="same", data_format='channels_first')(up8)
conv8 = Conv2D(3, (3, 64), activation="relu", padding="same", data_format='channels_first')(conv8)

up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=1)
conv9 = Conv2D(3, (3, 32), activation="relu", padding="same", data_format='channels_first')(up9)
conv9 = Conv2D(3, (3, 32), activation="relu", padding="same", data_format='channels_first')(conv9)



"""
#conv10 = Convolution2D(nb_classes,1, activation='softmax')(conv5)
flat = Flatten()(conv5)
dense1 = Dense(1024, activation="relu")(flat)
drp1 = Dropout(0.5)(dense1)
dense2 = Dense(1024, activation="relu")(drp1)
drp2 = Dropout(0.5)(dense2)
dense3 = Dense(nb_classes, activation="softmax")(drp1)

model = Model(input=inputs, output=dense3)
#model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=[ 'accuracy'])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:61: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [59]:
### LEARNING ###
from keras.optimizers import Adam, Adadelta
# First, use AdaDelta for some epochs because AdaMax gets stuck
model.compile(loss='categorical_crossentropy', 
              optimizer=Adadelta(),  
              metrics=["accuracy"])

In [60]:
# 20 epochs is sufficient
model.fit(X_train, Y_train, batch_size=128,
                    #nb_epoch=20, 
                    epochs=20,
                    validation_data=(X_val, Y_val),
                    verbose=1)

Train on 5026 samples, validate on 1257 samples
Epoch 1/20
5026/5026 [==============================] - 5s - loss: 4.1062 - acc: 0.0702 - val_loss: 4.0702 - val_acc: 0.0732
Epoch 2/20
5026/5026 [==============================] - 4s - loss: 3.9140 - acc: 0.0720 - val_loss: 3.7730 - val_acc: 0.0732
Epoch 3/20
5026/5026 [==============================] - 4s - loss: 3.7896 - acc: 0.0706 - val_loss: 3.7857 - val_acc: 0.0732
Epoch 4/20
5026/5026 [==============================] - 4s - loss: 3.7834 - acc: 0.0718 - val_loss: 3.7762 - val_acc: 0.0732
Epoch 5/20
5026/5026 [==============================] - 4s - loss: 3.7839 - acc: 0.0702 - val_loss: 3.7687 - val_acc: 0.0732
Epoch 6/20
5026/5026 [==============================] - 4s - loss: 3.7782 - acc: 0.0734 - val_loss: 3.7685 - val_acc: 0.0732
Epoch 7/20
5026/5026 [==============================] - 4s - loss: 3.7783 - acc: 0.0722 - val_loss: 3.7768 - val_acc: 0.0732
Epoch 8/20
5026/5026 [==============================] - 4s - loss: 3.7784 - a

In [39]:
# Parametrize the image augmentation class
datagen = ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.15,
    height_shift_range = 0.15,
    shear_range = 0.4,
    zoom_range = 0.3,                    
    channel_shift_range = 0.1)

In [40]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adamax',  
              metrics=["accuracy"])

# We want to keep the best model. This callback will store 
# in a file the weights of the model with the highest validation accuracy  
saveBestModel = ModelCheckpoint("best.kerasModelWeights", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True)

# Make the model learn using the image generator
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=128),
                samples_per_epoch=len(X_train),
                epochs=500, 
                validation_data=(X_val, Y_val),
                callbacks=[saveBestModel],
                verbose=1)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=500, steps_per_epoch=39, callbacks=[<keras.ca..., validation_data=(array([[[..., verbose=1)`
  from ipykernel import kernelapp as app


Epoch 1/500
39/39 [==============================] - 6s - loss: 3.7842 - acc: 0.0713 - val_loss: 3.7669 - val_acc: 0.0732
Epoch 2/500
39/39 [==============================] - 5s - loss: 3.7765 - acc: 0.0722 - val_loss: 3.7662 - val_acc: 0.0732
Epoch 3/500
39/39 [==============================] - 5s - loss: 3.7787 - acc: 0.0726 - val_loss: 3.7672 - val_acc: 0.0732
Epoch 4/500
39/39 [==============================] - 5s - loss: 3.7789 - acc: 0.0744 - val_loss: 3.7660 - val_acc: 0.0732
Epoch 5/500
27/39 [===================>..........] - ETA: 1s - loss: 3.7704 - acc: 0.0702

KeyboardInterrupt: 

In [12]:
# Predict the class (give the index in the one-hot vector of the most probable class)
Y_test_pred = model.predict_classes(X_test)

# Translate integers to character labels
vInt2label = np.vectorize(int2label)
Y_test_pred = vInt2label(Y_test_pred)

# Save the predicitions in Kaggle format
np.savetxt(path+"/CNN_pred.csv", np.c_[range(6284,len(Y_test_pred)+6284),Y_test_pred], delimiter=',', header = 'ID,Class', comments = '', fmt='%s')



6208/6220 [============================>.] - ETA: 0s

array([[[ 0.19527058],
        [ 0.1110196 ],
        [ 0.13943137],
        [ 0.22663529],
        [ 0.34954512],
        [ 0.42044312],
        [ 0.40160391],
        [ 0.36905882],
        [ 0.34546274],
        [ 0.28856862],
        [ 0.15297647],
        [ 0.15309413],
        [ 0.14388235],
        [ 0.12828235],
        [ 0.13422745],
        [ 0.13422745],
        [ 0.12408236],
        [ 0.14004706],
        [ 0.13288628],
        [ 0.11848235]],

       [[ 0.19286275],
        [ 0.19821568],
        [ 0.38222745],
        [ 0.60171372],
        [ 0.6007843 ],
        [ 0.5609569 ],
        [ 0.56600785],
        [ 0.5657059 ],
        [ 0.54178828],
        [ 0.53986669],
        [ 0.48180392],
        [ 0.35228235],
        [ 0.18111765],
        [ 0.13691765],
        [ 0.12448628],
        [ 0.12661961],
        [ 0.12823921],
        [ 0.16470589],
        [ 0.18260784],
        [ 0.14382352]],

       [[ 0.13287058],
        [ 0.26495296],
        [ 0.62138039],
       